In [241]:
!pip install pfrl@git+https://github.com/voidful/pfrl.git
!pip install textrl==0.2.13

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/voidful/pfrl.git to /tmp/pip-install-q8ppy5t8/pfrl_202a3af612d54315aba3e8bd3883333c
  Running command git clone --filter=blob:none --quiet https://github.com/voidful/pfrl.git /tmp/pip-install-q8ppy5t8/pfrl_202a3af612d54315aba3e8bd3883333c
  Resolved https://github.com/voidful/pfrl.git to commit 2ad3d51a7a971f3fe7f2711f024be11642990d61
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [243]:
from textrl import TextRLEnv,TextRLActor
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer, AutoModelWithLMHead , BlenderbotForConditionalGeneration
import logging
import sys
import pfrl
import torch

In [245]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [246]:
from transformers import AutoTokenizer, AutoModelWithLMHead
another_tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-emotion")
another_model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-emotion").to("cuda:0")

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1352: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [248]:
from transformers import AutoTokenizer, BlenderbotForConditionalGeneration
mname = "facebook/blenderbot-400M-distill"
model = BlenderbotForConditionalGeneration.from_pretrained(mname).to("cuda:0")
tokenizer = AutoTokenizer.from_pretrained(mname)

In [249]:
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# tokenizer = AutoTokenizer.from_pretrained("facebook/blenderbot-90M")
# model = AutoModelForSeq2SeqLM.from_pretrained("facebook/blenderbot-90M").to("cuda:0")
# model.eval()
# model.cuda()

In [250]:
UTTERANCE = ["What is your outlook on life?"]
print("Bot 1:",UTTERANCE)
for i in range(10):
    
    inputs = tokenizer(UTTERANCE, return_tensors="pt").to("cuda:0")
    reply_ids = model.generate(**inputs)
   
    UTTERANCE = tokenizer.batch_decode(reply_ids, skip_special_tokens=True)
    print("Bot",str(i%2),":",UTTERANCE)

Bot 1: ['What is your outlook on life?']
Bot 0 : [' My outlook is to be happy with who I am and to be content with where I am in life.']
Bot 1 : [" That's a good outlook to have. What do you do for a living, if you don't mind me asking?"]
Bot 0 : [" I work in a call center. It's not the most glamorous job in the world, but it pays the bills."]
Bot 1 : [' I used to work in one as well.  It was a lot of fun.  What do you do in your free time?']
Bot 0 : [' I like to play video games and hang out with my friends. What about you? Do you have any hobbies?']
Bot 1 : [" Video games are fun, but I don't have a lot of time for them."]
Bot 0 : [' I know what you mean, they can be very time consuming. What kind of games do you like to play?']
Bot 1 : [' I like RPGs and adventure games the most. What about you? Do you have any favorites?']
Bot 0 : [" I love adventure and RPG's as well. I have a lot of favorites."]
Bot 1 : [' Adventure is a great genre. What is your favorite RPG? I really like Dunge

In [251]:
def get_emotion(text):
    input_ids = tokenizer.encode(text + '</s>', return_tensors='pt')
    output = model.generate(input_ids=input_ids,max_length=200)
    dec = [tokenizer.decode(ids) for ids in output]
    label = dec[0]
    return label

In [252]:
dialogue_list = []

with open("/content/dialogues_text.txt", "r", encoding="utf-8") as file:
    for line in file:
        dialogue_parts = line.strip().split("__eou__")
        dialogue_list.extend(dialogue_parts)

observation_list = dialogue_list

In [253]:
# Check if the list contains empty strings
has_empty_strings = any(element.isspace() for element in observation_list)

if has_empty_strings:
    print("The list contains empty strings.")
else:
    print("The list does not contain empty strings.")

The list does not contain empty strings.


In [265]:
class MyRLEnv(TextRLEnv):
    def get_reward(self, input_item, predicted_list, finish): # predicted will be the list of predicted token
        reward = 0
        if finish:
            predicted_text = another_tokenizer.convert_tokens_to_string(predicted_list)
            # sentiment classifier
            if(get_emotion(predicted_text)[6:] == 'joy'):
                reward = -1000
            elif(get_emotion(predicted_text)[6:] == 'love'):
                reward = -1000
            elif(get_emotion(predicted_text)[6:] == 'surprise'):
                reward = -1000
            elif(get_emotion(predicted_text)[6:] == 'sadness'):
                reward = 100
            elif(get_emotion(predicted_text)[6:] == 'fear'):
                reward = 100
            elif(get_emotion(predicted_text)[6:] == 'anger'):
                reward = 100
        return reward

In [266]:
env = MyRLEnv(model, another_tokenizer, observation_input=observation_list,compare_sample=3)
actor = TextRLActor(env,model,tokenizer)
agent = actor.agent_ppo(update_interval=100, minibatch_size=3, epochs=10)

In [268]:
pfrl.experiments.train_agent_with_evaluation(
    agent,
    env,
    steps=300,
    eval_n_steps=None,
    eval_n_episodes=1,       
    train_max_episode_len=100,  
    eval_interval=10,
    outdir='elon_musk_dogecoin', 
)

(<textrl.actor.TextPPO at 0x7f55ae688910>,
 [{'average_value': -0.72880656,
   'average_entropy': 4.606346,
   'average_value_loss': 0.06618846761994064,
   'average_policy_loss': 0.013455330338329076,
   'n_updates': 334,
   'explained_variance': 0.9115965218817765,
   'eval_score': 0.0},
  {'average_value': -0.42905903,
   'average_entropy': 4.448599,
   'average_value_loss': 0.03760535953566432,
   'average_policy_loss': -0.001881647133268416,
   'n_updates': 668,
   'explained_variance': 0.2706570932651071,
   'eval_score': 0.0},
  {'average_value': -0.30766016,
   'average_entropy': 4.539911,
   'average_value_loss': 0.009971934825880453,
   'average_policy_loss': 0.005531531348824501,
   'n_updates': 1002,
   'explained_variance': 0.413468489106787,
   'eval_score': 0.0}])

In [271]:
UTTERANCE = ["What is your outlook on life?"]
print("Bot 1:",UTTERANCE)
for i in range(10):
    
    inputs = tokenizer(UTTERANCE, return_tensors="pt").to("cuda:0")
    reply_ids = model.generate(**inputs)
   
    UTTERANCE = tokenizer.batch_decode(reply_ids, skip_special_tokens=True)
    print("Bot",str(i%2),":",UTTERANCE)

Bot 1: ['What is your outlook on life?']
Bot 0 : [' My outlook is to be happy with who I am and to be content with where I am in life.']
Bot 1 : [" That's a good outlook to have. What do you do for a living, if you don't mind me asking?"]
Bot 0 : [" I work in a call center. It's not the most glamorous job in the world, but it pays the bills."]
Bot 1 : [' I used to work in one as well. It was not glamourous at all.']
Bot 0 : [" Yeah, it wasn't glamorous, but it was a good way to make some extra money."]
Bot 1 : [" That's true, I guess I was just a little envious of the guy who was able to do that."]
Bot 0 : [' Yeah, I know what you mean.  I wish I could do that too.  ']
Bot 1 : [" I know, right?  It's just so easy to get caught up in the moment."]
Bot 0 : [' Yes, it is. I wish I could go back in time and do it all over again.']
Bot 1 : [" I know, right?  It was so much fun.  I don't know what I would do without it."]
